# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [71]:
from os import getcwd
from ticktack.fitting import SingleFitter
from ticktack import load_presaved_model
from jax.numpy import array, pi, mean, arange

In [68]:
carbon_box_model = load_presaved_model(     # Generating the CarbobBoxModel using ticktack
    "Guttler14",                            # Selecting an arbitrary model 
    production_rate_units="atoms/cm^2/s"    # Units of the model
)

bayesian_model = SingleFitter(carbon_box_model)   # Fitting a model 
bayesian_model.prepare_function(model="simple_sinusoid") # Generating the simple sin model
bayesian_model.load_data(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv")   

mcmc_samples = bayesian_model.MarkovChainSampler(
    array([775., 1./12, pi/2., 81./12]),    # Initial guess of the parameters
    likelihood=bayesian_model.log_joint_simple_sinusoid,    
    args=(array([770., 0., -pi, 0.]), array([780., 5., pi, 15.])) # Upper and lower bounds
)   

Running burn-in...


100%|██████████| 500/500 [00:41<00:00, 11.94it/s]


Running production...


100%|██████████| 1000/1000 [01:27<00:00, 11.38it/s]


In [75]:
concentrations, final_concentrations = carbon_box_model.run(
    time_out=arange(750, 800), 
    oversample=108, 
    production=bayesian_model.production, 
    steady_state_production=1.88,
    args = mean(mcmc_samples, axis=0)
)

(DeviceArray([[ 134.89141868,  709.60871501, 1191.75522842, ...,
               1808.40827592,  665.96078961, 7348.8025404 ],
              [ 134.89220763,  709.60905505, 1191.75522861, ...,
               1808.40827592,  665.96078961, 7348.8025404 ],
              [ 134.89295021,  709.60937891, 1191.75522917, ...,
               1808.40827592,  665.96078961, 7348.8025404 ],
              ...,
              [ 135.20188588,  711.95605407, 1196.02857066, ...,
               1808.91523182,  666.13008388, 7348.83757498],
              [ 135.20764908,  711.95884778, 1196.02676052, ...,
               1808.915419  ,  666.13014662, 7348.83758899],
              [ 135.21341968,  711.96166807, 1196.02495542, ...,
               1808.91560608,  666.13020932, 7348.837603  ]],            dtype=float64),
 DeviceArray([1.34891419e+02, 7.09608715e+02, 1.19175523e+03,
              3.97248139e+00, 4.51603166e+04, 1.55628969e+02,
              6.34361837e+02, 4.23744476e+02, 1.80840828e+03,
           